# Flask와 Rest API

## 6. Flask와 크롤링 결과 연동

### 네이버 OPEN API를 활용한 책 검색

In [14]:
import requests

# 인증 정보
client_id = "ZFjdNkku5Fu9ZDAVe7xX"
client_secret = "nr4OwLElUH"

# 기본 url 정보
url = "https://openapi.naver.com/v1/search/book.json"

# 검색할 책 이름 입력
q = "파이썬"
# url 호출 시 전달할 요청 변수 정보
params = {"query": q,      # 책이름
          "display": 5,    # 검색할 건수
          "sort": "count"} # 정렬순서

# requests 라이브러리를 이용한 책 검색 api 호출
# get 방식으로 호출(url)/ 요청 변수 전달(params)/ 인증 정보 및 인코딩 정보 전달(header)
response = requests.get(url=url, params=params,
                        headers={"X-Naver-Client-Id": client_id,
                                 "X-Naver-Client-Secret": client_secret,
                                 "Content-Type": "application/json; charset=utf-8"})

# 호출 처리 상태 정보 recode 변수에 할당
rescode = response.status_code

if (rescode == 200):
    # 호출 처리 상태가 정상(200) 일 경우리턴 받은 책 정보 저장
    data = response.json()
else:
    print("Error Code:", rescode)

# Naver 책 검색 API 응답 중 실제 책 아이템 데이터 추출 및 출력
print(data)
# item_list = data['items']
# # print(item_list)
# # print(len(item_list))

# # 검색한 도서를 리스트 형태로 저장하여 리턴하기
# book_list = []
# for item in item_list:
#     # print(item['title'].replace('<b>','').replace('</b>',''))
#     book_list.append(item["title"].replace('<b>','').replace('</b>','')+'<br>')

# print(book_list)


{'lastBuildDate': 'Tue, 16 Mar 2021 16:22:01 +0900', 'total': 1250, 'start': 1, 'display': 5, 'items': [{'title': '혼자 공부하는 <b>파이썬</b> (<b>파이썬</b> 최신 버전 반영)', 'link': 'http://book.naver.com/bookdb/book_detail.php?bid=15028688', 'image': 'https://bookthumb-phinf.pstatic.net/cover/150/286/15028688.jpg?type=m1&udate=20210127', 'author': '윤인성', 'price': '18000', 'discount': '16200', 'publisher': '한빛미디어', 'pubdate': '20190610', 'isbn': '1162241888 9791162241882', 'description': '1:1 과외하듯 배우는 <b>파이썬</b> 프로그래밍 자습서(<b>파이썬</b> 최신 버전 반영)\n27명의 베타리더 검증으로, ‘함께 만든’ 입문자 맞춤형 도서이 책은 독학으로 프로그래밍 언어를 처음 배우려는 입문자가, 혹은 <b>파이썬</b>을 배우려는 입문자가 ‘꼭 필요한 내용을 제대로’ 학습할 수 있도록 구성했다. ‘무엇을’... '}, {'title': 'Do it! 점프 투 <b>파이썬</b>', 'link': 'http://book.naver.com/bookdb/book_detail.php?bid=15052904', 'image': 'https://bookthumb-phinf.pstatic.net/cover/150/529/15052904.jpg?type=m1&udate=20200910', 'author': '박응용', 'price': '18800', 'discount': '16920', 'publisher': '이지스퍼블리싱', 'pubdate': '20190620', 'isbn': '1163030910 97

### Flask와 네이버 책 검색 연동
- 1) flask 구동 파이썬 파일
- 2) index.html

In [13]:
from flask import Flask, request, jsonify
import pprint
import requests
import json

app = Flask(__name__)

# 루트에 접근할 경우 
@app.route('/', methods=['GET'])
def index():
    with open('index.html', 'rb') as f:
        return f.read()
    
@app.route('/api', methods=['GET', 'POST'])
def search_book():
    request_json = request.json
    
    # 검색할 책 이름을 index.html에서 받아오기
    q = request.args.get('q', '')
    if q == '':
        return jsonify("조회할 책 이름을 입력해주세요!!")
    print("q=", q)   

    # 네이버 OPEN API 인증 정보
    client_id = "ZFjdNkku5Fu9ZDAVe7xX"
    client_secret = "nr4OwLElUH"

    # 네이버 도서명 검색 url 정보
    url = "https://openapi.naver.com/v1/search/book.json"

    # url 호출 시 전달할 요청 변수 정보
    params = {"query": q,      # 책이름
              "display": 5,    # 조회 건수
              "sort": "count"} # 정렬 순서

    # requests 라이브러리를 이용한 책 검색 api 호출
    # get 방식으로 호출(url)/ 요청 변수 전달(params)/ 인증 정보 및 인코딩 정보 전달(header)
    response = requests.get(url=url, params=params,
                            headers={"X-Naver-Client-Id": client_id,
                                     "X-Naver-Client-Secret": client_secret,
                                     "Content-Type": "application/json; charset=utf-8"})
    # 호출 처리 상태 정보 recode 변수에 할당
    rescode = response.status_code

    if (rescode == 200):
        # 호출 처리 상태가 정상(200) 일 경우리턴 받은 책 정보 저장
        #print(response.json())
        data = response.json()
    else:
        print("Error Code:", rescode)

    # Naver 책 검색 API 응답 중 실제 책 아이템 데이터 추출 및 출력
    item_list = data['items']
    
    

    # 검색한 도서를 리스트 형태로 저장하여 리턴하기
    book_list = []
    for item in item_list:
        book_list.append(item["title"].replace('<b>','').replace('</b>','')+'<br>') 
    
    
    # 결과값 리턴하기
    res = ''
    for i, book in enumerate(book_list, start=1):
        res += str(i) + ':' + book
    return jsonify(res)
    
    

if __name__ == "__main__":    
    # 서버 실행하기
    app.run(host="127.0.0.1", port="8000")


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)


### index.html

In [ ]:
<DOCTYPE html>
<html>
<head>
  <meta charset="utf-8">
</head>
<body>
      <h2>네이버 책 검색하기</h2>
      <div>
          <textarea id="q" rows="2" cols="40"></textarea>
          <br>
          <button id="qButton">검색</button>
          <div id="result"></div>
      </div>
      <script>
          const qs = (q) => document.querySelector(q)
          window.onload = () => {
              const q = qs('#q')
              const qButton = qs('#qButton')
              const result = qs('#result')
              // 조회 버튼을 눌렀을 때 
              qButton.onclick = () => {
                  result.innerHTML = "..."
                  // URL 생성하기 
                  const api = "/api?q=" + encodeURIComponent(q.value)
                  // API에 접근하기
                  fetch(api).then((res) => {
                      return res.json() // JSON 응답
                  }).then((data) => {
                      // 결과를 화면에 출력하기 
                      result.innerHTML = 
                            data
                  })
              }
          }
      </script>
      <style>
          #result {
              padding: 5px;
              font-size: 1.2em;
              color: red;
          }
          
          #q {
              background-color: #fffff0;
          }
      </style>
</body>
</html>

## 7. 프론트엔드와 백엔드 Flask 로 한번에 구현해보기

### 폴더 구조
```
flask_test 폴더
    /login.py
    /templates 폴더
        /login.html
    static 폴더
```

### login.html 
```html
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Document</title>
  </head>
  <body>
    <form action="/login" method="get">
      <p>Enter Name:</p>
      <p><input type="text" name="user_name" /></p>
      <p><input type="submit" value="submit" /></p>
    </form>
  </body>
</html>
```

### login.py 파이썬 코드
- GET 요청으로 받는 url은 아래와 같이 코드 작성

```python
from flask import Flask, jsonify, request
app = Flask(__name__)


@app.route('/login')
def login():
    username = request.args.get('user_name')
    if username == 'grace':
        return_data = {'auth':'success'}
    else:
        return_data = {'auth':'failed'}
    return jsonify(return_data)

if __name__ == '__main__':
    app.run(host="127.0.0.1", port="5000")
```

### Rest API 요청시 파라미터/파라미터값 넣기
- HTTP 의 요청 방식 중, 가장 많이 사용되는 방식이 GET 방식
  - GET 방식에서는 URI 상에 파라미터와 파라미터 값을 넣을 수 있음
    - 규칙: URL?파라미터1=파라미터1값&파라미터2=파라미터2값 
    - URL 이후 첫 파라미터 이름 전에 ? 를 표시하고, 추가 파라미터가 있을 시에는 & 표시를 해야 함
    - (예) http://localhost:8080/login?user_name=grace&pwd=1111

### Flask 로 정적 웹페이지 로드하기 
- 프론트엔드 페이지도 flask 로 보여줄 수 있음

- flask render_template(HTML파일명): HTML 파일 전송하기
  - HTML파일은 flask 가 실행되는 하위 폴더인 templates 폴더 안에 위치해야 함

```python
@app.route('/html_test')
def hello_html():
    # html file은 templates 폴더에 위치해야 함
    return render_template('login.html')
```

### login.py 파일 업데이트
- flask 의 render_template 함수 추가
- @app.route('html_test') 추가

```python
from flask import Flask, jsonify, request, render_template
app = Flask(__name__)

@app.route('/login')
def login():
    username = request.args.get('user_name')
    if username == 'grace':
        return_data = {'auth': 'success'}
    else:
        return_data = {'auth': 'failed'}
    return jsonify(return_data)


@app.route('/html_test')
def hello_html():
    # html file은 templates 폴더에 위치해야 함
    return render_template('login.html')

if __name__ == '__main__':
    app.run(host="127.0.0.1", port="8000")
```

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="3em" style="font-weight:bold;color:#3f8dbf;">코드 실행하기</font><br>
1) login.py 실행 후 python login.py<br>
2) http://127.0.0.1:8000/html_test 웹페이지 브라우저로 실행
</div>

In [5]:
from flask import Flask, jsonify, request, render_template
app = Flask(__name__)

@app.route('/login')
def login():
    username = request.args.get('user_name')
    if username == 'grace':
        return_data = {'auth': 'success'}
    else:
        return_data = {'auth': 'failed'}
    return jsonify(return_data)


@app.route('/html_test')
def hello_html():
    # html file은 templates 폴더에 위치해야 함
    return render_template('login.html')

if __name__ == '__main__':
    app.run(host="127.0.0.1", port="8000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Jan/2021 12:22:17] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [24/Jan/2021 12:22:23] "GET /grace HTTP/1.1" 404 -
127.0.0.1 - - [24/Jan/2021 12:22:31] "GET /read_html HTTP/1.1" 404 -
